In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_wiki= requests.get(wiki_url).text
soup = BeautifulSoup(raw_wiki,'html5lib')

In [3]:
table=soup.find('table')

In [4]:
Postcode= []
Borough= []
Neighbourhood = []

In [5]:
x=0 #counter
for table_cell in table.find_all('td'):
    
    x = x+1
    if (x==1):
        postal_code = table_cell.text
    elif (x==2):
        borough = table_cell.text
    elif (x==3):
        neighborhoods_data = table_cell.text
        neighborhoods = neighborhoods_data.strip('\n')
        x=0 #reset counter
                             
        if borough == 'Not assigned':            
            borough = []
            neighborhoods_data=[]
        else:
            if neighborhoods == 'Not assigned':
                neighborhoods = borough
             
            Postcode.append(postal_code)
            Borough.append(borough)
            Neighbourhood.append(neighborhoods)

In [6]:
data=[]

for postcode_unique_element in set(Postcode):
    post_value = ''
    borough_value_var = ''
    neighbourhood_value = ''
    
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            post_value = postcode_element;
            borough_value_var = Borough[postcode_idx]
            
            if neighbourhood_value == '': 
                neighbourhood_value = Neighbourhood[postcode_idx]
            else:
                neighbourhood_value = neighbourhood_value + ', ' + Neighbourhood[postcode_idx]
    
    data.append([post_value,borough_value_var,neighbourhood_value])

df = pd.DataFrame(np.array(data),columns=['PostalCode', 'Borough', 'Neighborhood'])

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M4B,East York,"Woodbine Gardens, Parkview Hill"
1,M3C,North York,"Flemingdon Park, Don Mills South"
2,M3K,North York,"CFB Toronto, Downsview East"
3,M2M,North York,"Newtonbrook, Willowdale"
4,M2J,North York,"Fairview, Henry Farm, Oriole"


In [8]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['PostalCode', 'Latitude', 'Longitude']

df_latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_join = pd.merge(df, df_latlng, on=['PostalCode'], how='inner')
df_join.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
1,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
2,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
3,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
5,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
6,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
7,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
8,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
9,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
